##  22.07.16

### Dataset, DataLoader
데이터 샘플을 처리하는 코드는 지저분하고 유지보수가 어려울 수 있기 때문에  
가독성을 높이고 모듈화를 위해서 데이터셋 코드를 모델 학습 코드로부터 분리하는 것이 이상적이다.  
Dataset은 샘플과 정답을 저장하고,  
DataLoader는 Dataset을 샘플에 쉽게 접근할 수 있도록 iterable 객체로 감싼다.  

### 사용자 정의 Dataset만들기
사용자 정의 Dataset 클래스는 반드시 3개 함수를 구현해야 한다.  
`__init__`, `__len__`, `__getitiem__`  
아래에서는 getitem을 통해 인덱스에 해당하는 이미지를 불러오고 해당 이미지와 맞는 label을 csv에서 가져온다.  

In [ ]:
import os

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import matplotlib.pyplot as plt
import pandas as pd

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


### DataLoader로 학습용 데이터 준비하기
모델을 학습할 때 일반적으로 샘플들을 미니배치로 전달하고, 매 에폭마다 데이터를 다시 섞어서 과적합을 막고,  
파이썬의 멀티프로세싱을 이용해 데이터 검색 속도를 높이려고 한다.  
Dataloader는 간단한 API로 이러한 과정들을 추상화한 이터러블 객체이다.  

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)